<a href="https://colab.research.google.com/github/sciencebyAJ/oet_gf_ti/blob/main/time_integration_gap_filling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sciencebyAJ/oet_gf_ti.git

Cloning into 'oet_gf_ti'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 95 (delta 27), reused 11 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (95/95), 17.81 MiB | 5.00 MiB/s, done.
Resolving deltas: 100% (27/27), done.
Updating files: 100% (16/16), done.


In [2]:
cd oet_gf_ti/

/content/oet_gf_ti


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import glob
import numpy as np
import scipy.stats as stats
import scipy.optimize as opt
from scipy.interpolate import Akima1DInterpolator, CubicSpline, KroghInterpolator
import requests
import datetime
import matplotlib.pyplot as plt
import json
from google.colab import userdata

In [7]:
import src.time_integration as ti
import src.stats as stats

In [8]:
def interp_var(in_df,varname = 'clear_sky_EToF',refeto = 'site'):
    all_df = in_df.copy()
    all_df['xs']=np.arange(0,all_df.shape[0])+1
    all_df['x'] = all_df['xs'].mask(np.isnan(all_df[varname]), np.nan)
    all_df['week']=all_df.index.isocalendar().week
    # linear interpolation
    all_df[varname+'_linear']=all_df[varname].interpolate('linear',limit=32,limit_area='inside', limit_direction='both')
    # akima interpolation (e.g. rolling spline)
    x = np.array(all_df['x'])[~np.isnan(all_df['x'])]
    y = np.array(all_df[varname])[~np.isnan(all_df[varname])]
    xs = np.array(all_df['xs'])
    y_akima = Akima1DInterpolator(x, y, method="akima")(xs)
    all_df[varname+'_Akima']=y_akima
    all_df[varname+'_Akima']=all_df[varname+'_Akima'].mask(np.isnan(all_df[varname+'_linear']), np.nan)
    # climatology interpolation
    df_clim = all_df[[varname,'x']].copy()
    df_clim[varname+'rm']=df_clim[varname].rolling(32,1,center=True).mean()
    df_nonans = df_clim[[varname+'rm','x']].dropna()
    X = np.array(df_nonans.x)
    fit = np.polyfit(X, df_nonans[varname+'rm'], 1)
    fit_fn = np.poly1d(fit)
    all_df[varname+'linear_trend']= fit_fn(xs)
    all_df[varname+'iav']=df_clim[varname+'rm']-all_df[varname+'linear_trend']
    serClimModel = all_df[varname+'iav'].rolling(32,1,center=True).mean().groupby([all_df.index.isocalendar().week]).mean()
    # serClimModel = all_df[varname].rolling(32,1,center=True).mean().groupby([all_df.index.isocalendar().week]).mean()
    var_clim_dict = serClimModel.to_dict()
    all_df[varname+'_clim']=all_df['week'].map(var_clim_dict)
    all_df[varname+'_noclim']=all_df[varname]-all_df[varname+'_clim']
    all_df[varname+'noclim_fill'] = all_df[varname+'_noclim'].interpolate(method='linear', limit=32,limit_area='inside', limit_direction='both')
    all_df[varname+'_clim_fill'] =all_df[varname+'noclim_fill']+all_df[varname+'_clim']

    # now conert back to ETo
    if refeto == 'gridMET':
      all_df[varname+'_linear_ET']=all_df[varname+'_linear']*all_df['eto']
      all_df[varname+'_clim_fill_ET']=all_df[varname+'_clim_fill']*all_df['eto']
      all_df[varname+'_Akima_ET']=all_df[varname+'_Akima']*all_df['eto']
    elif refeto == 'site':
      try:
        all_df[varname+'_linear_ET']=all_df[varname+'_linear']*all_df['ASCE_ETo']
        all_df[varname+'_clim_fill_ET']=all_df[varname+'_clim_fill']*all_df['ASCE_ETo']
        all_df[varname+'_Akima_ET']=all_df[varname+'_Akima']*all_df['ASCE_ETo']
      except:
        all_df[varname+'_linear_ET']=all_df[varname+'_linear']*all_df['gridMET_ETo']
        all_df[varname+'_clim_fill_ET']=all_df[varname+'_clim_fill']*all_df['gridMET_ETo']
        all_df[varname+'_Akima_ET']=all_df[varname+'_Akima']*all_df['gridMET_ETo']
      pass

    else:
      all_df[varname+'_linear_ET']=all_df[varname+'_linear']*all_df['gridMET_ETo']
      all_df[varname+'_clim_fill_ET']=all_df[varname+'_clim_fill']*all_df['gridMET_ETo']
      all_df[varname+'_Akima_ET']=all_df[varname+'_Akima']*all_df['gridMET_ETo']

    return all_df


verbose=True

time_ags = ['MAM','JJA','SON','DJF','Annual','Growing']
time_ag_dict ={'MAM':[3,4,5],
              'JJA':[6,7,8],
              'SON':[9,10,11],
              'DJF':[12,1,2],
              'Annual':[1,2,3,4,5,6,7,8,9,10,11,12],
              'Growing':[4,5,6,7,8,9,10]}



In [14]:
SITE_PATH   = 'data/'
OUT_PATH   = 'results/'
openETapikey = userdata.get('open_et_api_key')
drive_ti_outpath = userdata.get('drive_ti_outath')

all_stats_df = pd.DataFrame(columns=['Site_id','Obs_freq','Obs_freq_rep','Fill_Method','Temp','Bias', 'MAE', 'RMSE', 'R2', 'KT'])

for filename in glob.glob('data/US-MM*.csv'):
  print(filename)
  try:
    tower_i = ti.get_tower_data(filename.split('/')[-1], SITE_PATH, OUT_PATH, openETapikey,debug=False)
    site_avail=True
  except:
    site_avail=False
    print('\t'+'site not available')
  if site_avail:
    all_df = tower_i.site_all_df.copy()
    all_df = interp_var(all_df,varname = 'clear_sky_EToF')
    all_df['openetof']=all_df['et']/all_df['eto']
    all_df = interp_var(all_df,varname = 'openetof')

    for i in [8,16,32,64]:
      for j in np.arange(i):
        out_list = ['freq_'+str(i), 'EToF_filt_'+str(i)+'_'+str(j)]
        varname_ij = f'EToF_filtered{str(i)}_{str(j)}'
        if verbose:
          print('\t'+varname_ij)
        all_df_ij = all_df.copy()
        all_df_ij = interp_var(all_df_ij,varname = varname_ij)
        if j ==0:
          all_df_ij.to_csv(OUT_PATH+'tables/'+tower_i.site_id+'_'+varname_ij+'data_for_figs.csv')
          all_df_ij.to_csv(drive_ti_outpath+'/'+tower_i.site_id+'_'+varname_ij+'data_for_figs.csv')

        for fmod in ['linear','Akima','clim_fill']:
          varname_ijm = varname_ij+'_'+fmod+'_ET'
          for temp_ag in time_ags:
            all_df_ij_temp = all_df_ij.loc[all_df_ij.index.month.isin(time_ag_dict[temp_ag])]
            try:
              out_stats = stats.get_summary_stats(all_df_ij_temp[varname_ijm], all_df_ij_temp['ET_corr'])
              all_stats_df.loc[len(all_stats_df.index)]=[tower_i.site_id]+out_list+[fmod,temp_ag]+out_stats
            except:
              print('\t\t'+varname_ijm+' not all stats able to be computed')
              continue
    all_stats_df.loc[all_stats_df['Site_id']==tower_i.site_id].to_csv(OUT_PATH+'tables/'+tower_i.site_id+'_stats.csv')
    all_stats_df.loc[all_stats_df['Site_id']==tower_i.site_id].to_csv(drive_ti_outpath+'/'+tower_i.site_id+'_stats.csv')
    plt.style.use('seaborn-v0_8')
    plt.figure(figsize=(6,2.5))
    tower_i.site_all_df.ET_corr.plot(color='lightgray',label='')
    tower_i.site_all_df.et.plot(style='.',label='ET$_{OpenET}$')
    tower_i.site_all_df.clear_sky_ET.plot(style='.',label='ET$_{EC_{clear sky}}$')
    tower_i.site_all_df.ET_corr16_0.plot(style='.',label='ET$_{EC_{16}}$')
    tower_i.site_all_df.ET_corr32_0.plot(style='.',label='ET$_{EC_{32}}$')
    tower_i.site_all_df.ET_corr64_0.plot(style='.',label='ET$_{EC_{64}}$')
    plt.title(tower_i.site_id)
    plt.legend(frameon=False,ncol=1,bbox_to_anchor=(1,1))
    plt.xlabel('')
    plt.ylabel('ET (mm/day)')
    plt.tight_layout()
    plt.savefig(OUT_PATH+'figures/'+tower_i.site_id+'_ET_clear_sky.png')

    plt.style.use('ggplot')
    plt.figure(figsize=(6,2.5))
    varname1 = 'clear_sky_EToF'
    varname = 'openetof'
    # tower_i.site_all_df.ASCE_ETo.plot(color='gray',label='',lw=5)
    tower_i.site_all_df.ET_corr.plot(style='.',color='lightgray',label='')
    all_df['clear_sky_EToF'+'_linear_ET'].plot(style='.',label='EC lin')
    all_df[varname+'_linear_ET'].plot(style='.',label='OpenET lin')
    all_df['clear_sky_EToF'+'_Akima_ET'].plot(style='.',label='Akima')
    all_df[varname+'_clim_fill_ET'].plot(style='.',c='purple',markersize=6,label=varname +' Clim Fill')
    tower_i.site_all_df.et.plot(style='.',label='ET$_{OpenET}$')
    tower_i.site_all_df.clear_sky_ET.plot(style='.',label='ET$_{EC_{clear sky}}$')
    plt.title(tower_i.site_id)
    plt.legend(frameon=False,ncol=1,bbox_to_anchor=(1,1))
    plt.xlabel('')
    plt.ylabel('ET (mm/day)')
    plt.tight_layout()
    plt.savefig(OUT_PATH+'figures/'+tower_i.site_id+'_interp_method_comp.png')

all_stats_df.to_csv(OUT_PATH + 'tables/' + 'all_site_stats.csv')
all_stats_df.to_csv(drive_ti_outpath + '/' + 'all_site_stats.csv')



data/US-MMS_daily_data.csv
	site not available


In [142]:
# temp_ag = 'SON'
# for meth in ['linear','Akima','clim_fill']:
#   print()
#   print(meth)
#   for f in [8,16,32]:
#     print()
#     print(f)
#     print(all_stats_df.loc[(all_stats_df['Obs_freq']==f'freq_{f}')&(all_stats_df['Fill_Method']==meth)&(all_stats_df['Temp']==temp_ag)][['Bias','MAE']].mean())

In [137]:
# for temp_ag in time_ags:
#   for mod in ['linear','Akima','clim_fill']:
#     for i in [8]:
#         print(mod,temp_ag, 'freq', i)
#         print(all_stats_df.loc[(all_stats_df['Fill_Method']==mod) & (all_stats_df['Obs_freq']==f'freq_{i}') & (all_stats_df['Temp']==temp_ag)][['Bias','MAE','RMSE','R2']].mean())
#         print('\n\n')


In [138]:
# all_stats_df.loc[(all_stats_df['Fill_Method']=='linear')&(all_stats_df['Obs_freq']=='freq_16')]
# all_stats_df.loc[(all_stats_df['Fill_Method']=='linear')&(all_stats_df['Obs_freq']=='freq_32')]

In [141]:
# # varname = 'clear_sky_EToF'
# fig,axs = plt.subplots(3,1,figsize=(3,8),sharex=True)
# ones = np.arange(-1,10,0.1)
# axs[0].plot(ones,ones,'k--')
# all_df.plot.scatter(x='ET_corr',y=varname1+'_clim_fill_ET',ax=axs[0])
# axs[1].plot(ones,ones,'k--')
# all_df.plot.scatter(x='ET_corr',y=varname1+'_linear_ET',ax=axs[1])
# axs[2].plot(ones,ones,'k--')
# all_df.plot.scatter(x='ET_corr',y=varname+'_linear_ET',ax=axs[2])
# axs[2].set_xlabel('ET$_{obs}$ (mm/day)')

In [139]:
# print('\n\n')
# print(varname1,' linear')
# print(all_df['ET_corr'].mask(np.isnan(all_df[varname1+'_linear_ET']), np.nan).mean())
# a = stats.get_summary_stats(all_df[varname1+'_linear_ET'],all_df['ET_corr'])
# print(a)

# print('\n\n')
# print(varname,' linear')
# print(all_df['ET_corr'].mask(np.isnan(all_df[varname+'_linear_ET']), np.nan).mean())
# b = stats.get_summary_stats(all_df[varname+'_linear_ET'],all_df['ET_corr'])
# print(b)
# print('\n\n')
# print(varname, 'clim')
# print(all_df['ET_corr'].mask(np.isnan(all_df[varname+'_clim_fill_ET']), np.nan).mean())
# c = stats.get_summary_stats(all_df[varname+'_clim_fill_ET'],all_df['ET_corr'])

# print('\n\n')
# print(varname, 'Akima')
# print(all_df['ET_corr'].mask(np.isnan(all_df[varname+'_Akima_ET']), np.nan).mean())
# d = stats.get_summary_stats(all_df[varname+'_Akima_ET'],all_df['ET_corr'])


In [140]:
# for i in np.arange(8):
#   print(i)
#   plt.figure(figsize = (6,3))
#   all_df.EToF_filtered.plot(c='black')
#   all_df[f'EToF_filtered8_{i}'].plot(style='o',markersize=4)
#   # all_df[f'etof8_'{str(i)}]=all_df[i::8]
#   # all_df[i::16].EToF_filtered.plot(style='o',markersize=3)
#   # all_df[i::32].EToF_filtered.plot(style='o',markersize=2)
#   plt.title(str(i))